In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Admin\AppData\Local\Temp\ipykernel_50316\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [3]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [4]:
# Your code
from sklearn.model_selection import train_test_split

# Suppose 'texts' is your input (email texts) and 'labels' is your target (e.g., 0 or 1 for fraud)
texts = data['text']
labels = data['label']

# Vectorize the text data
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X = vectorizer.fit_transform(texts)

# Split into 80% training and 20% test sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

print("Train set size:", X_train.shape)
print("Test set size:", X_test.shape)

Train set size: (800, 1000)
Test set size: (200, 1000)


In [5]:
print(data.columns)

Index(['text', 'label'], dtype='object')


## Data Preprocessing

In [6]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [7]:
import re

def clean_html(raw_html):
    # 1. Remove inline JavaScript/CSS: <script>...</script> and <style>...</style>
    cleaned = re.sub(r'<(script|style).*?>.*?</\1>', '', raw_html, flags=re.DOTALL)

    # 2. Remove HTML comments: <!-- ... -->
    cleaned = re.sub(r'<!--.*?-->', '', cleaned, flags=re.DOTALL)

    # 3. Remove all remaining HTML tags: <...>
    cleaned = re.sub(r'<.*?>', '', cleaned)

    
    return cleaned

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [8]:
# Your code
def preprocess_text(text):

    # Remove special characters and numbers
    text = re.sub(r'[^a-z\s]', ' ', text)
    
    # Remove single characters (e.g., "a", "b")
    text = re.sub(r'\b[a-zA-Z]\b', '', text)
    
    # Remove single characters from the start (e.g., " b something" → "something")
    text = re.sub(r'^[a-zA-Z]\s+', '', text)
    
    # Remove prefixed 'b' (in case of byte string decoding leftovers)
    text = re.sub(r'^b\s+', '', text)
    
    # Substitute multiple spaces with single space
    text = re.sub(r'\s+', ' ', text).strip()
     # Convert to string and lowercase
    text = str(text).lower()
    
   # Tokenize
    words = text.split()

    # Remove stopwords and stem words
    words = [snowball.stem(word) for word in words if word not in stopwords.words('english')]
    
    return ' '.join(words)

In [10]:
data['text'] = data['text'].apply(clean_html)
data['cleaned_text'] = data['text'].apply(preprocess_text)

## Now let's work on removing stopwords
Remove the stopwords.

In [11]:
# Your code
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))

def remove_stopwords(text):
    words = text.split()
    filtered = [word for word in words if word not in stop_words]
    return ' '.join(filtered)

In [12]:
data['no_stopwords'] = data['cleaned_text'].apply(remove_stopwords)

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [ ]:
# Your code
#1. Import NLTK’s WordNet Lemmatizer:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')  # For full language support

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
#2. Define a function to lemmatize text:
def lemmatize_text(text):
    words = text.split()
    lemmatized = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized)

In [ ]:
#3. Apply it to your no-stopword text:
data['lemmatized_text'] = data['no_stopwords'].apply(lemmatize_text)

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [29]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer

# Separate the messages
spam_texts = data[data['label'] == 1]['lemmatized_text']
ham_texts = data[data['label'] == 0]['lemmatized_text']

# Drop empty values
spam_texts = spam_texts.dropna()
spam_texts = spam_texts[spam_texts.str.strip() != '']

ham_texts = ham_texts.dropna()
ham_texts = ham_texts[ham_texts.str.strip() != '']

# Vectorize spam
vectorizer = CountVectorizer()
spam_bow = vectorizer.fit_transform(spam_texts)
spam_words = vectorizer.get_feature_names_out()
spam_counts = spam_bow.sum(axis=0).A1
spam_freq = list(zip(spam_words, spam_counts))
top_spam_words = sorted(spam_freq, key=lambda x: x[1], reverse=True)[:10]

# Vectorize ham
vectorizer = CountVectorizer()
ham_bow = vectorizer.fit_transform(ham_texts)
ham_words = vectorizer.get_feature_names_out()
ham_counts = ham_bow.sum(axis=0).A1
ham_freq = list(zip(ham_words, ham_counts))
top_ham_words = sorted(ham_freq, key=lambda x: x[1], reverse=True)[:10]

# Print results
print("Top 10 Spam Words:")
for word, count in top_spam_words:
    print(f"{word}: {count}")

print("\nTop 10 Ham Words:")
for word, count in top_ham_words:
    print(f"{word}: {count}")

Top 10 Spam Words:
money: 824
account: 714
fund: 666
bank: 485
nbsp: 475
transact: 456
transfer: 440
countri: 423
busi: 408
contact: 393

Top 10 Ham Words:
pm: 107
would: 105
call: 103
bama: 102
work: 102
time: 82
percent: 81
ecretari: 80
tate: 78
merican: 69


## Extra features

In [31]:
# Define the feature patterns
money_simbol_list = "|".join(["euro", "dollar", "pound", "€", "$"])
suspicious_words = "|".join([
    "free", "cheap", "sex", "money", "account", "bank", "fund",
    "transfer", "transaction", "win", "deposit", "password"
])

# Add extra features to your main data
data['money_mark'] = data['lemmatized_text'].str.contains(money_simbol_list, case=False, regex=True) * 1
data['suspicious_words'] = data['lemmatized_text'].str.contains(suspicious_words, case=False, regex=True) * 1
data['text_len'] = data['lemmatized_text'].apply(len)

# View updated DataFrame
print(data[['lemmatized_text', 'money_mark', 'suspicious_words', 'text_len']].head())

                                     lemmatized_text  money_mark  \
1                                                ill           1   
2  os heryl email dozen memo aiti weekend plea pr...           1   
3  ear ir adam know propos might surpris emerg nu...           1   
4                                                fyi           1   
5  sure bottom line need special secur code get l...           1   

   suspicious_words  text_len  
1                 0         3  
2                 0        98  
3                 1      1067  
4                 0         3  
5                 0       268  


## How would work the Bag of Words with Count Vectorizer concept?

In [32]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [ ]:
# Your code
#Import the Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#Initialize the Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=1000)

In [ ]:
#Fit & Transform Your Text Data
X_tfidf = tfidf_vectorizer.fit_transform(data['lemmatized_text'])

In [ ]:
#Print the Shape of the Vectorized Data
print("TF-IDF Matrix Shape:", X_tfidf.shape)

TF-IDF Matrix Shape: (940, 1000)


## And the Train a Classifier?

In [37]:
# Your code
y = data['label']  # 0 for ham, 1 for spam

In [38]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [39]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [40]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9521276595744681

Classification Report:
               precision    recall  f1-score   support

           0       0.92      1.00      0.96       108
           1       1.00      0.89      0.94        80

    accuracy                           0.95       188
   macro avg       0.96      0.94      0.95       188
weighted avg       0.96      0.95      0.95       188


Confusion Matrix:
 [[108   0]
 [  9  71]]


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code